In [5]:
# BETTER WAY 03 - bytes, str, unicode

print(b'foo')
print(type(b'foo'))

import os

def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value # str 인스턴스
repr(to_str(b'foo'))

b'foo'
<class 'bytes'>


"'foo'"

In [16]:
# orthogonality #
##################################

def cal(a,b,c):
    return a+b+c

def show(price):
    return f"$ {price}"

def final(a,b,c, fmt=str):
    return fmt(cal(a,b,c))

final(10, 0.2, 0.5, fmt=show)


<function __main__.kkk(a, b)>

In [12]:
def lastNmae_and_FirstName(*Nmaes):
    return [[name, *name[1:10]] for name in Nmaes]


lastNmae_and_FirstName('이천수', '산다라박')

[['이천수', '천', '수'], ['산다라박', '다', '라', '박']]